In [ ]:
#Model.py
#We use paded convolutions
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF

class DoubleConv(nn.module):
  def __init__ (self, in_channels, out_channels):
    super(DoubleConv, self).__init__()
    self.conv = nn.sequential(
        nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias = False), #(in channel, out channel, kernel size, stride, padding(1 for same padding), bias)
        nn.BatchNorm2d(out_channels), # as we use batchnorm the bias will cansel so we dont use bias in our convolution
        nn.ReLU(inplace = True),
        nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias = False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace = True),
    )
  def forward(self, x):
    return self.conv(x)

class unet (nn.Module):
  def __init__ (self, in_channels=3, out_channels= 1, features=[64, 128, 256, 512]): #we have RGB immages so in chnnaels is 3 and a binary classifier so out channel is 1 features is the number of featurs in each stage convolution layer
    super(unet, self).__init__()
    self.ups = nn.ModuleList()
    self.downs = nn.ModuleList()
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    #down part of u-net
    for feature in features:
      self.downs.append(DoubleConv(in_channel, feature))
      in_channels = feature

    #up part of u-net
    for feature in reversed(features):
      self.ups.append(nn.ConvTranspose2d(feature*2, feature, kernel_size = 2, stride = 2)) #up conves in paper
      self.ups.append(DoubleConv(feature*2, feature))

    self.bottleneck = DoubleConv(features[-1], features[-1]*2)
    self.finalconv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forrward(self, x):
      skip_connections = []
      for down in downs:
        x = downs(x)
        skip_connections.append(x)
        x = self.pool(x)
        
        
      x = self.bottleneck(x)
      skip_connections = skip_connections[::-1]
      for idx in range(0, len(self.ups), 2):
        x = self.ups[idx](x) #this is the up sampling applyed on x
        skip_connection = skip_connections[idx//2]
        
        if x.shape != skip_connection.shape:
          x= TF.resize(x, size = skip_connection[2:]) #we just get its height and width

        concat_skip = torch.cat((skip_connection, x), dim = 1) #we add them along the channel dimention (batch, channel, height, width)
        x = self.ups[idx+1](concat_skip) #this is doubleconves appled on concatinat data (x + skip_connection)

      return self.finalconv(x)